In [51]:
import pandas as pd
from Bio import SeqIO
from io import StringIO
import os
import subprocess
import ray
from subprocess import Popen, PIPE, STDOUT

In [52]:
input_fasta= "/home/infosebi/Documents/Programmieren/MastersProject/clusters.fasta"
output_folder= "/home/infosebi/Documents/Programmieren/MastersProject/clusters"
mafft_folder="/home/infosebi/Documents/Programmieren/MastersProject/mafft_out"

In [53]:
cluster_df = pd.read_csv('reclusters.tsv', sep='\t', header=None)

In [54]:
prot_lists= []
prot_old = ''
for index, row in cluster_df.iterrows():
    prot = row[0]
    if prot != prot_old:
        if index > 0:
            prot_lists.append(prot_list)
        prot_list = []
        prot_list.append(row[1])
    else:
        prot_list.append(row[1])
    prot_old = prot    

In [55]:
seq_dict = {}
for record in SeqIO.parse("clusters.fasta", "fasta"):
        seq_dict[record.id]=record

In [56]:
cluster_list = []
for cluster in prot_lists:
    helper_list = []
    for protein in cluster:
        helper_list.append(seq_dict[protein])
    cluster_list.append(helper_list)


In [86]:
ray.shutdown()
ray.init()

2023-11-03 14:20:01,967	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.8
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [87]:
@ray.remote
def maf_tree(cluster):
    if len(cluster) >= 2:
        out_handle = StringIO()
        SeqIO.write(cluster, out_handle, "fasta")
        fasta_data = out_handle.getvalue()
        #output_file = output_folder+f'/cl_{cluster[0].id}.fasta'
        #mafft = subprocess.Popen(["mafft", "--auto", "--thread", "8", "input.fasta", ">", output_file], shell=True, text=True)
        #mafft = Popen(["mafft"], text=True,stdout=PIPE, stdin=PIPE, stderr=PIPE)
        #stdout_data, stderr_data = mafft.communicate(input=fasta_data)
        mafft = subprocess.Popen(["mafft", "--auto", "--thread", "8", "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout_data, stderr_data = mafft.communicate(input=fasta_data)
        output_file = f'{output_folder}/cl_{cluster[0].id}.fasta'
        with open(output_file, 'w') as output_handle:
            output_handle.write(stdout_data)
    if len(cluster) >= 3:
            subprocess.run(f"fasttree < {mafft_folder}/alignment_{cluster[0].id}.fasta >  {mafft_folder}/{cluster[0].id}.tre",shell=True)
#def maf_tree(cluster):
#    SeqIO.write(cluster,output_folder+f'/cl_{cluster[0].id}.fasta','fasta')
#    if len(cluster) >= 2:
#        #subprocess popen -> write fasta file to process
#        #get rid of uknown description
#        mafft = Popen(["mafft", "--auto"],stdout=PIPE, stdin=PIPE, stderr=PIPE)
#        stdout_data = mafft.communicate(input=cluster)[0]
#        print(stdout_data)
#        #subprocess.run(f"mafft --auto {output_folder}/cl_{cluster[0].id}.fasta > {mafft_folder}/alignment_{cluster[0].id}.fasta",shell=True)
#        if len(cluster) >= 3:
#            subprocess.run(f"fasttree < {mafft_folder}/alignment_{cluster[0].id}.fasta >  {mafft_folder}/{cluster[0].id}.tre",shell=True)

In [88]:
futures = [maf_tree.remote(cluster) for cluster in cluster_list]

(maf_tree pid=146125) FastTree Version 2.1.11 Double precision (No SSE3)
(maf_tree pid=146125) Alignment: standard input
(maf_tree pid=146125) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
(maf_tree pid=146125) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
(maf_tree pid=146125) TopHits: 1.00*sqrtN close=default refresh=0.80
(maf_tree pid=146125) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
(maf_tree pid=146125) Initial topology in 0.01 seconds
(maf_tree pid=146125) Refining topology: 0 rounds ME-NNIs, 2 rounds ME-SPRs, 0 rounds ML-NNIs
(maf_tree pid=146125) Total branch-length 0.000 after 0.01 sec
(maf_tree pid=146125) Total time: 0.01 seconds Unique: 1/3 Bad splits: 0/0
(maf_tree pid=146120) ML-NNI round 1: LogLk = -545.680 NNIs 0 max delta 0.00 Time 0.01
(maf_tree pid=146120) Switched to using 20 rate categories (CAT approximation)
(maf_tree pid=146120) Rate categories were divided by 0.634 so that average rate = 1

(maf_tree pid=146125)       0.10 seconds: Site likelihoods with rate category 6 of 20
(maf_tree pid=146125) ML-NNI round 3: LogLk = -2552.225 NNIs 0 max delta 0.00 Time 0.21 (final)
(maf_tree pid=146125)       0.21 seconds: ML Lengths 1 of 1 splits
(maf_tree pid=146124)       0.21 seconds: ML NNI round 1 of 5, 1 of 4 splits
(maf_tree pid=146124)       1.12 seconds: ML NNI round 3 of 5, 1 of 4 splits
(maf_tree pid=146121) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 13x across cluster]
(maf_tree pid=146121) Alignment: standard input [repeated 13x across cluster]
(maf_tree pid=146121) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 13x across cluster]
(maf_tree pid=146121) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 13x across cluster]
(maf_tree pid=146121) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 13x across cluster]
(maf_tree pid=146121) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 

(maf_tree pid=146122)       0.83 seconds: ML NNI round 3 of 5, 1 of 4 splits [repeated 5x across cluster]
(maf_tree pid=146119) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 13x across cluster]
(maf_tree pid=146119) Alignment: standard input [repeated 13x across cluster]
(maf_tree pid=146119) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 13x across cluster]
(maf_tree pid=146119) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 13x across cluster]
(maf_tree pid=146119) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 13x across cluster]
(maf_tree pid=146119) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 13x across cluster]
(maf_tree pid=146119) Initial topology in 0.01 seconds [repeated 13x across cluster]
(maf_tree pid=146119) Refining topology: 4 rounds ME-NNIs, 2 rounds ME-SPRs, 2 rounds ML-NNIs [repeated 13x across cluster]
(maf_tree pid=146119) Total branch-length 

(maf_tree pid=146125) 
(maf_tree pid=146120)       0.25 seconds: ME NNI round 1 of 17, 1 of 16 splits [repeated 2x across cluster]
(maf_tree pid=146119)       0.25 seconds: ML NNI round 1 of 6, 1 of 5 splits [repeated 7x across cluster]
(maf_tree pid=146122) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 4x across cluster]
(maf_tree pid=146122) Alignment: standard input [repeated 4x across cluster]
(maf_tree pid=146122) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 4x across cluster]
(maf_tree pid=146122) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 4x across cluster]
(maf_tree pid=146122) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 4x across cluster]
(maf_tree pid=146122) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 4x across cluster]
(maf_tree pid=146119) Initial topology in 0.01 seconds [repeated 3x across cluster]
(maf_tree pid=146119) Refining topology: 1

(maf_tree pid=146123)      11.81 seconds: ML Lengths 1 of 28 splits
(maf_tree pid=146118) ML-NNI round 1: LogLk = -7442.495 NNIs 7 max delta 9.15 Time 7.08
(maf_tree pid=146119) Turning off heuristics for final round of ML NNIs
(maf_tree pid=146123) Total branch-length 0.782 after 0.02 sec
(maf_tree pid=146122) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 5x across cluster]
(maf_tree pid=146122) Alignment: standard input [repeated 5x across cluster]
(maf_tree pid=146122) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 5x across cluster]
(maf_tree pid=146122) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 5x across cluster]
(maf_tree pid=146122) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 5x across cluster]
(maf_tree pid=146122) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 5x across cluster]
(maf_tree pid=146122) Initial topology in 0.01 seconds [repeated 5x acro

...

(maf_tree pid=146121) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 16x across cluster]
(maf_tree pid=146121) Alignment: standard input [repeated 16x across cluster]
(maf_tree pid=146121) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 16x across cluster]
(maf_tree pid=146121) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 16x across cluster]
(maf_tree pid=146121) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 16x across cluster]
(maf_tree pid=146121) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 16x across cluster]
(maf_tree pid=146121) Initial topology in 0.00 seconds [repeated 16x across cluster]
(maf_tree pid=146121) Refining topology: 6 rounds ME-NNIs, 2 rounds ME-SPRs, 3 rounds ML-NNIs [repeated 16x across cluster]
(maf_tree pid=146121) Total branch-length 0.046 after 0.00 sec [repeated 16x across cluster]
(maf_tree pid=146119) Total time: 0.64 seconds Unique: 

(maf_tree pid=146124) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 18x across cluster]
(maf_tree pid=146124) Alignment: standard input [repeated 18x across cluster]
(maf_tree pid=146124) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 18x across cluster]
(maf_tree pid=146124) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 18x across cluster]
(maf_tree pid=146124) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 18x across cluster]
(maf_tree pid=146124) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 18x across cluster]
(maf_tree pid=146124) Initial topology in 0.00 seconds [repeated 18x across cluster]
(maf_tree pid=146124) Refining topology: 0 rounds ME-NNIs, 2 rounds ME-SPRs, 0 rounds ML-NNIs [repeated 18x across cluster]
(maf_tree pid=146124) Total branch-length 0.000 after 0.00 sec [repeated 18x across cluster]
(maf_tree pid=146124) Total time: 0.00 seconds Unique: 

(maf_tree pid=146124)       0.32 seconds: ML NNI round 1 of 6, 1 of 7 splits
(maf_tree pid=146124)       0.86 seconds: Site likelihoods with rate category 1 of 20
(maf_tree pid=146124)       0.97 seconds: Site likelihoods with rate category 10 of 20
(maf_tree pid=146124)       1.09 seconds: Site likelihoods with rate category 18 of 20
(maf_tree pid=146119) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 10x across cluster]
(maf_tree pid=146119) Alignment: standard input [repeated 10x across cluster]
(maf_tree pid=146119) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 10x across cluster]
(maf_tree pid=146119) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 10x across cluster]
(maf_tree pid=146119) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 10x across cluster]
(maf_tree pid=146119) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 10x across cluster]
(maf_tree pid=146119

(maf_tree pid=146125)       0.65 seconds: Site likelihoods with rate category 1 of 20 [repeated 11x across cluster]
(maf_tree pid=146125) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 10x across cluster]
(maf_tree pid=146125) Alignment: standard input [repeated 10x across cluster]
(maf_tree pid=146125) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 10x across cluster]
(maf_tree pid=146125) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 10x across cluster]
(maf_tree pid=146125) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 10x across cluster]
(maf_tree pid=146125) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 10x across cluster]
(maf_tree pid=146125) Initial topology in 0.01 seconds [repeated 10x across cluster]
(maf_tree pid=146125) Refining topology: 11 rounds ME-NNIs, 2 rounds ME-SPRs, 6 rounds ML-NNIs [repeated 10x across cluster]
(maf_tree pid=146125) Total bra

(maf_tree pid=146125) FastTree Version 2.1.11 Double precision (No SSE3) [repeated 9x across cluster]
(maf_tree pid=146125) Alignment: standard input [repeated 9x across cluster]
(maf_tree pid=146125) Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000 [repeated 9x across cluster]
(maf_tree pid=146125) Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1 [repeated 9x across cluster]
(maf_tree pid=146125) TopHits: 1.00*sqrtN close=default refresh=0.80 [repeated 9x across cluster]
(maf_tree pid=146125) ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories [repeated 9x across cluster]
(maf_tree pid=146125) Initial topology in 0.00 seconds [repeated 9x across cluster]
(maf_tree pid=146125) Refining topology: 0 rounds ME-NNIs, 2 rounds ME-SPRs, 0 rounds ML-NNIs [repeated 9x across cluster]
(maf_tree pid=146125) Total branch-length 0.000 after 0.00 sec [repeated 9x across cluster]
(maf_tree pid=146125) Total time: 0.00 seconds Unique: 1/5 Bad s

In [84]:
def maf_tree(cluster):
    if len(cluster) >= 2:
        out_handle = StringIO()
        SeqIO.write(cluster, out_handle, "fasta")
        fasta_data = out_handle.getvalue()
        #output_file = output_folder+f'/cl_{cluster[0].id}.fasta'
        #mafft = subprocess.Popen(["mafft", "--auto", "--thread", "8", "input.fasta", ">", output_file], shell=True, text=True)
        #mafft = Popen(["mafft"], text=True,stdout=PIPE, stdin=PIPE, stderr=PIPE)
        #stdout_data, stderr_data = mafft.communicate(input=fasta_data)
        mafft = subprocess.Popen(["mafft", "--auto", "--thread", "8", "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout_data, stderr_data = mafft.communicate(input=fasta_data)
        output_file = f'{output_folder}/cl_{cluster[0].id}.fasta'
        with open(output_file, 'w') as output_handle:
            output_handle.write(stdout_data)
    if len(cluster) >= 3:
            subprocess.run(f"fasttree < {mafft_folder}/alignment_{cluster[0].id}.fasta >  {mafft_folder}/{cluster[0].id}.tre",shell=True)
#for cluster in cluster_list:
maf_tree(cluster_list[5000])

In [ ]:
#create zip files from genome
#include mafft_alignments
#include tree folder